In [1]:
import json
import multiprocessing
import os
import random
import requests
import time
from typing import List, Literal

from bs4 import BeautifulSoup
import httpx
import pandas as pd
from pydantic import BaseModel, Field
from tqdm import tqdm

from src.config import settings
# from src.http_client import HTTPXClient

‘금융위원회 - 정책마당 - 정책일반 - 금융정책’의 PDF 파일들 다운로드
- 목록 URL : `https://www.fsc.go.kr/po010101?curPage={pageNum}&srchCtgry=1`
- 상세 페이지 URL: `https://www.fsc.go.kr/po010101/{itemNum}?srchCtgry=1&curPage={pageNum}&srchKey=&srchText=&srchBeginDt=&srchEndDt=`
    - itemNum은 목록에 보이는 번호가 아닌 구분 값 (ex. “84155”)
- 파일 다운로드 URL: `https://www.fsc.go.kr/comm/getFile?srvcId=BBSTY1&upperNo={itemNum}&fileTy=ATTACH&fileNo={fileNo}`

# 1. 목차 페이지 파싱

## 1-1. 목차 페이지 Get

In [2]:
def get_page(page_no: int):
    url = f"https://www.fsc.go.kr/po010101?curPage={page_no}&srchCtgry=1"
    headers = {"Content-Type": "application/json"}
    with httpx.Client() as client:
        response = client.get(url, headers=headers)
    response.raise_for_status()
    return response.text

In [3]:
page = get_page(1)

## 1-2. 파싱

In [4]:
class ItemFile(BaseModel):
    no: int
    name: str
    extension: Literal["pdf", "hwp", "hwpx", "doc", "docx"]
    
class ItemDetail(BaseModel):
    page_no: int = Field(-1)
    item_no: str = Field("")
    item_id: str = Field("")
    title: str = Field("")
    date: str = Field("")
    files: List[ItemFile] = Field(list())

In [5]:
def parse_list_page(html_content: str, page_no: int = -1):
    soup = BeautifulSoup(html_content, 'html.parser')
    items = []
    for item in soup.find_all('div', class_='inner'):
        # 아이템 번호 찾기
        count_div = item.find('div', class_='count')
        if count_div:
            # 리스트상 번호
            item_number = count_div.text.strip()

            # 날짜 찾기
            date_div = item.find('div', class_='day')
            date_value = date_div.text.strip() if date_div else None

            # href 찾기
            link_div = item.find('div', class_='subject').find('a')
            href_value = link_div['href'] if link_div else None
            title_value = link_div.text.strip() if link_div else "No Title"
            
            # 고유 번호 추출
            unique_id = href_value.split('/')[-1].split('?')[0] if href_value else None
            
            # 파일 리스트를 저장할 리스트
            files = []
            # 첨부파일 존재 여부 및 파일명, fileNo 찾기
            file_elements = item.find_all('div', class_='file-list')
            for file_elem in file_elements:
                file_name = file_elem.find('span', class_='name').text.strip()
                file_no = file_elem.find('span', class_='ico download').find('a')['href']
                file_no = file_no.split('fileNo=')[-1]  # fileNo 추출
                
                file_extension = file_name.rsplit(".", 1)[-1]
                if file_extension not in ["pdf", "hwp", "hwpx", "doc", "docx"]:
                    continue
                file = ItemFile(
                    no=file_no,
                    name=file_name,
                    extension=file_extension
                )
                files.append(file)
            item_detail = ItemDetail(
                page_no=page_no,
                item_no=item_number,
                item_id=unique_id,
                title=title_value,
                date=date_value,
                files=files
            )
            items.append(item_detail)
    return items

In [6]:
page_items = parse_list_page(page, page_no=1)

In [7]:
page_items

[ItemDetail(page_no=1, item_no='2718', item_id='84155', title='[보도참고] 신용보증기금이 P-CBO를 직접 발행하여 기업의 비용부담을 약 50bp 절감합니다 - 「신용보증기금법」 개정안, 국회 본회의 통과', date='2025-03-13', files=[ItemFile(no=1, name='250313(보도참고) 신용보증기금이 P-CBO를 직접 발행하여 기업의 비용부담을 약 50bp 절감합니다.pdf', extension='pdf'), ItemFile(no=2, name='250313(보도참고) 신용보증기금이 P-CBO를 직접 발행하여 기업의 비용부담을 약 50bp 절감합니다.hwp', extension='hwp'), ItemFile(no=3, name='250313(보도참고) 신용보증기금이 P-CBO를 직접 발행하여 기업의 비용부담을 약 50bp 절감합니다.hwpx', extension='hwpx')]),
 ItemDetail(page_no=1, item_no='2717', item_id='84130', title='[보도자료] 민･관이 함께 건전한 가상자산시장  조성을 위해 노력해나가겠습니다. - 금융위원회, 가상자산 업계·전문가 간담회 개최', date='2025-03-12', files=[ItemFile(no=1, name='250312(보도자료) 민･관이 함께 건전한 가상자산시장을 조성해나가겠습니다. - 금융위원회  가상자산 업계·전문가 간담회 개최 -.pdf', extension='pdf'), ItemFile(no=2, name='250312(보도자료) 민･관이 함께 건전한 가상자산시장을 조성해나가겠습니다. - 금융위원회  가상자산 업계·전문가 간담회 개최 -.hwp', extension='hwp'), ItemFile(no=3, name='250312(보도자료) 민･관이 함께 건전한 가상자산시장을 조성해나가겠습니다. - 금융위원회  가상자산 업계·전문가 간담회 개최 -.hwpx', extension

In [8]:
items = []
failed_idxs = []

for page_no in tqdm(range(1, 100)):
    try:
        page = get_page(page_no)
        page_items = parse_list_page(page, page_no=page_no)
        items.extend(page_items)
    except Exception as e:
        print(f"Page {page_no} Failed - {str(e)}")
        failed_idxs.append(page_no)
        
    x = random.randint(0, 10)
    time.sleep(0.1*x)

100%|██████████| 99/99 [03:14<00:00,  1.96s/it]


In [9]:
items_with_files = list(filter(lambda x: len(x.files)>0, items))
len(items), len(items_with_files)

(990, 990)

In [10]:
from datetime import datetime
dates = [
    datetime.strptime(x.date, "%Y-%m-%d")
    for x in items
]
min(dates), max(dates)

(datetime.datetime(2021, 5, 20, 0, 0), datetime.datetime(2025, 3, 13, 0, 0))

In [11]:
with open(os.path.join(settings.data_dir, "retrieval_dataset/2503-01-korean-finance/kr-fsc_policy_metadata.json"), "w") as f:
    # item_dicts = []
    # for item in item_dicts:
        
    f.write(json.dumps(
        [x.dict() for x in items],
        indent=4,
        ensure_ascii=False
    ))

/var/folders/wj/0c7skj2154q4844jqxlw3yxr0000gn/T/ipykernel_31661/3180321472.py:6: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  [x.dict() for x in items],


# Filter Files

In [12]:
class FileDetail(BaseModel):
    page_no: int = Field(-1)
    item_no: str = Field("")
    item_id: str = Field("")
    # item_name: str = Field("")
    item_date: str = Field("")
    item_title: str = Field("")
    no: int = Field(...)
    name: str = Field("")
    extension: Literal["pdf", "hwp", "hwpx", "doc", "docx"]

In [15]:
pdf_file_details = []

for item in items:
    for file in item.files:
        if file.extension=="pdf":
            detail = FileDetail(
                page_no=item.page_no,
                item_no=item.item_no,
                item_id=item.item_id,
                item_date=item.date,
                item_title=item.title,
                no=file.no,
                name=file.name,
                extension=file.extension
            )
            pdf_file_details.append(detail)
len(pdf_file_details)

1332

In [16]:
pdf_file_details[:3]

[FileDetail(page_no=1, item_no='2718', item_id='84155', item_date='2025-03-13', item_title='[보도참고] 신용보증기금이 P-CBO를 직접 발행하여 기업의 비용부담을 약 50bp 절감합니다 - 「신용보증기금법」 개정안, 국회 본회의 통과', no=1, name='250313(보도참고) 신용보증기금이 P-CBO를 직접 발행하여 기업의 비용부담을 약 50bp 절감합니다.pdf', extension='pdf'),
 FileDetail(page_no=1, item_no='2717', item_id='84130', item_date='2025-03-12', item_title='[보도자료] 민･관이 함께 건전한 가상자산시장  조성을 위해 노력해나가겠습니다. - 금융위원회, 가상자산 업계·전문가 간담회 개최', no=1, name='250312(보도자료) 민･관이 함께 건전한 가상자산시장을 조성해나가겠습니다. - 금융위원회  가상자산 업계·전문가 간담회 개최 -.pdf', extension='pdf'),
 FileDetail(page_no=1, item_no='2717', item_id='84130', item_date='2025-03-12', item_title='[보도자료] 민･관이 함께 건전한 가상자산시장  조성을 위해 노력해나가겠습니다. - 금융위원회, 가상자산 업계·전문가 간담회 개최', no=4, name='[붙임] 가상자산 업계 전문가 간담회_김소영 부위원장 모두발언_수정.pdf', extension='pdf')]

In [17]:
with open(os.path.join(settings.data_dir, "retrieval_dataset/2503-01-korean-finance/kr-fsc_pdf_file_metadata.json"), "w") as f:
    # item_dicts = []
    # for item in item_dicts:
        
    f.write(json.dumps(
        [x.dict() for x in pdf_file_details],
        indent=4,
        ensure_ascii=False
    ))

/var/folders/wj/0c7skj2154q4844jqxlw3yxr0000gn/T/ipykernel_31661/342585427.py:6: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  [x.dict() for x in pdf_file_details],


In [18]:
df = pd.DataFrame.from_dict([x.dict() for x in pdf_file_details])

/var/folders/wj/0c7skj2154q4844jqxlw3yxr0000gn/T/ipykernel_31661/2305275716.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  df = pd.DataFrame.from_dict([x.dict() for x in pdf_file_details])


In [19]:
df.head()

,page_no,item_no,item_id,item_date,item_title,no,name,extension
0,1,2718,84155,2025-03-13,[보도참고] 신용보증기금이 P-CBO를 직접 발행하여 기업의 비용부담을 약 50bp...,1,250313(보도참고) 신용보증기금이 P-CBO를 직접 발행하여 기업의 비용부담을 ...,pdf
1,1,2717,84130,2025-03-12,[보도자료] 민･관이 함께 건전한 가상자산시장 조성을 위해 노력해나가겠습니다. -...,1,250312(보도자료) 민･관이 함께 건전한 가상자산시장을 조성해나가겠습니다. - ...,pdf
2,1,2717,84130,2025-03-12,[보도자료] 민･관이 함께 건전한 가상자산시장 조성을 위해 노력해나가겠습니다. -...,4,[붙임] 가상자산 업계 전문가 간담회_김소영 부위원장 모두발언_수정.pdf,pdf
3,1,2716,84129,2025-03-12,[보도자료] 2025년 2월중 가계대출 동향(잠정) - ’25.2월중 全...,1,250312(보도자료) 2025년 2월중 가계대출 동향(잠정).pdf,pdf
4,1,2715,84123,2025-03-12,[보도자료] “나도 모르게 개설되는 계좌” 이제는 사전에 차단할 수 있습니다. -...,1,250311(보도자료) “나도 모르게 개설되는 계좌” 이제는 사전에 차단할 수 있습...,pdf
